# Primary outcomes
**P1.  Cost per 1,000 patients for all 18 pre-specified “low-priority” treatments combined.**

**P2. Total items per 1000 across all 18 low priority treatments.**


In [1]:
import os
import requests
import pandas as pd
import numpy as np

from analysis import compute_regression

import logging
logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.ERROR)

GBQ_PROJECT_ID = '620265099307'
DUMMY_RUN = False  # Useful for testing; set to false when doing real analysis

# Set dates of baseline and follow-up periods
baseline_start = '2018-04-01'       # baseline start
mid_start = '2018-10-01'            # month after end of baseline period
followup_start = '2019-04-01'       # follow-up start
post_followup_start = '2019-10-01'  # month after end of follow-up period

all_measures = ['lpcoprox', 'lpdosulepin', 'lpdoxazosin', 
                'lpfentanylir', 'lpglucosamine', 'lphomeopathy', 
                'lplidocaine', 'lpliothyronine', 'lplutein', 
                'lpomega3', 'lpoxycodone', 'lpperindopril', 
                'lprubefacients', 'lptadalafil', 'lptramadolpara', 
                'lptravelvacs', 'lptrimipramine','lpherbal']
definition_url = (
    "https://raw.githubusercontent.com/ebmdatalab/openprescribing/"
    "{commit}/openprescribing/frontend/management/commands/measure_definitions/"
    "{measure}.json")
commit_for_measure_definitions = "6f949660fee06401102136926eaba075d963511d"

#import herbal list manually due to different construction of query based on a separate file
herbal_bnf_list = pd.read_csv(os.path.join('..','data','herbal_list.csv'), usecols=["bnf_code"])
herbal_bnf_list = tuple(herbal_bnf_list["bnf_code"])

In [2]:
# Import data from BigQuery
# (Specifically, per-measure cost/items numerators, and population denominators)
if DUMMY_RUN and os.path.exists(os.path.join('..','data','all_measure_data.csv')):
    rawdata = pd.read_csv(os.path.join('..','data','all_measure_data.csv')).drop(['Unnamed: 0'], axis=1)
else:
    rawdata = pd.DataFrame()
    sql_template = open("measure.sql", "r").read()
    for measure in all_measures:
        if measure == "lpherbal":
            where_condition = f"(bnf_code IN {herbal_bnf_list})"
        else:
            measure_definition = requests.get(definition_url.format(
                commit=commit_for_measure_definitions, measure=measure)).json()
            where_condition = " ".join(measure_definition['numerator_where'])
        sql = sql_template.format(
            date_from=baseline_start, 
            date_to=post_followup_start, 
            where_condition=where_condition)
        df = pd.read_gbq(sql, GBQ_PROJECT_ID, dialect='standard')
        df["month"] = pd.to_datetime(df.month)
        df["measure"] = measure
        rawdata = rawdata.append(df)
rawdata.head(1)

,month,pct_id,items,cost,denominator,measure
0,2018-10-01 00:00:00+00:00,00C,4,392.31788,108.634,lpcoprox


In [3]:
rawdata.to_csv(os.path.join('..','data','all_measure_data.csv'))

In [4]:
# Aggregate across all measures 
data = rawdata.groupby(["pct_id", "month"]).agg(
    {'items':'sum', 'cost': 'sum', 'denominator':'first'}).reset_index()
data = data.rename(columns={"cost": "numerator"})
data['calc_value'] = data['numerator'] / data['denominator']
data.head(2)

,pct_id,month,items,numerator,denominator,calc_value
0,00C,2018-04-01 00:00:00+00:00,487,16142.79865,108.273,149.093483
1,00C,2018-05-01 00:00:00+00:00,470,16842.40218,108.299,155.517615


In [5]:
# select data only for the baseline and follow-up periods

conditions = [
    (data['month'] >= post_followup_start),
    (data['month'] >= followup_start),
    (data['month'] >= mid_start),
    (data['month'] >= baseline_start),
    (data['month'] < baseline_start)]

choices = ['after', 'follow-up', 'mid', 'baseline', 'before']
data['period'] = np.select(conditions, choices, default='0')
# Restrict to columns of interest
data = data[["pct_id", "period", "month", "numerator", "denominator", "items"]]
data = data.loc[
    (data['period'] == "baseline") | (data['period'] == "follow-up")
].set_index(["pct_id", "period", "month"])

data.head(3)

numerator  denominator  items
pct_id period   month                                                     
00C    baseline 2018-04-01 00:00:00+00:00  16142.79865      108.273    487
                2018-05-01 00:00:00+00:00  16842.40218      108.299    470
                2018-06-01 00:00:00+00:00  17039.99407      108.356    481

In [6]:
# group measurements for each CCG for each period
agg_6m = data.groupby(["pct_id", "period"]).agg(
    {"numerator": "sum", "items": "sum", "denominator": "mean"})
agg_6m.head()

numerator  items  denominator
pct_id period                                     
00C    baseline    99880.62004   2805   108.418333
       follow-up   91403.31905   2413   108.911500
00D    baseline   222129.62651   6802   292.331833
       follow-up  214533.63594   5527   293.096833
00J    baseline   287709.89820   6868   259.248000

In [7]:
### CCGs that have been allocated to the RCT 
rct_ccgs = pd.read_csv(os.path.join('..','data','randomisation_group.csv'))


# Joint Team information (which CCGs work together in Joint Teams)
team = pd.read_csv(os.path.join('..','data','joint_teams.csv'))

# Map CCGs to Joint Teams
rct_ccgs = rct_ccgs.merge(team, on="joint_team", how="left")

# Fill blank ccg_ids from joint_id column, so even CCGs not in Joint Teams 
# have a value for joint_id
rct_ccgs["pct_id"] = rct_ccgs["ccg_id"].combine_first(rct_ccgs["joint_id"])
rct_ccgs = rct_ccgs[["joint_id", "allocation", "pct_id"]]

# Combine CCG/Joint Team info with measure data
rct_ccgs = rct_ccgs.merge(agg_6m.reset_index(), on="pct_id", how="left")
rct_ccgs.head(3)


,joint_id,allocation,pct_id,period,numerator,items,denominator
0,01X,con,01X,baseline,320249.69042,9672,197.704333
1,01X,con,01X,follow-up,273113.29526,6961,198.111167
2,99K,con,99K,baseline,271463.76501,6309,170.713667


In [8]:
# aggregate up to Joint team groups
# sum both numerator and population denominator across joint teams 
rct_agg_6m = rct_ccgs\
             .groupby(["joint_id", "allocation", "period"])\
             .sum()\
             .unstack()\
             .reset_index()
# Rename columns which have awkward names resulting from the unstack operation
rct_agg_6m.columns = rct_agg_6m.columns.map('_'.join).map(lambda x: x.strip("_"))
# Create binary "intervention" column for later regression
rct_agg_6m['intervention'] = rct_agg_6m.allocation.map({'con': 0, 'I': 1})
rct_agg_6m.head(3)

,joint_id,allocation,numerator_baseline,numerator_follow-up,items_baseline,items_follow-up,denominator_baseline,denominator_follow-up,intervention
0,00J,I,287709.89820,257801.09800,6868,5622,259.248000,261.553833,1
1,00Y,con,252412.36670,164723.89551,11098,4759,255.777833,258.976833,0
2,01F,con,207092.79661,146517.32527,5809,4646,131.572667,132.790333,0


In [9]:
# calculate aggregated measure values for baseline and followup pareiods
rct_agg_6m["baseline_calc_value"] = (
    rct_agg_6m.numerator_baseline / rct_agg_6m.denominator_baseline)
rct_agg_6m["follow_up_calc_value"] = (
    rct_agg_6m["numerator_follow-up"] / rct_agg_6m["denominator_follow-up"])
rct_agg_6m["baseline_items_thou"] = (
    rct_agg_6m.items_baseline / rct_agg_6m.denominator_baseline)
rct_agg_6m["follow_up_items_thou"] = (
    rct_agg_6m["items_follow-up"] / rct_agg_6m["denominator_follow-up"])

rct_agg_6m.head(3)

,joint_id,allocation,numerator_baseline,numerator_follow-up,items_baseline,items_follow-up,denominator_baseline,denominator_follow-up,intervention,baseline_calc_value,follow_up_calc_value,baseline_items_thou,follow_up_items_thou
0,00J,I,287709.89820,257801.09800,6868,5622,259.248000,261.553833,1,1109.786375,985.652149,26.492008,21.494619
1,00Y,con,252412.36670,164723.89551,11098,4759,255.777833,258.976833,0,986.842227,636.056490,43.389217,18.376161
2,01F,con,207092.79661,146517.32527,5809,4646,131.572667,132.790333,0,1573.980386,1103.373428,44.150507,34.987487



# Baseline characteristics

In [10]:

# Obtain list sizes including older-age proportion

sql = '''SELECT pct_id, AVG(over_65)/1000 AS over_65, AVG(total_list_size) AS total_list_size --average over 6-month period
FROM (SELECT prac.ccg_id AS pct_id, month,
SUM(male_65_74+ male_75_plus+ female_65_74+ female_75_plus) AS over_65, -- sum up to CCGs
SUM(total_list_size) AS total_list_size
FROM ebmdatalab.hscic.practice_statistics_all_years stats
LEFT JOIN ebmdatalab.research.practices_2019_09 prac ON stats.practice = prac.code
WHERE month between '{date_from}' and '{date_to}'
GROUP BY pct_id, month)
GROUP BY pct_id'''

sql = sql.format(date_from=baseline_start, date_to=mid_start)

#sql = open("measure.sql", "r").read()
pop = pd.read_gbq(sql, GBQ_PROJECT_ID, dialect='standard')
pop.to_csv(os.path.join('..','data','ccg_populations.csv'))
pop.head()


,pct_id,over_65,total_list_size
0,00C,21.225000,108449.142857
1,00D,60.118429,292381.428571
2,00J,49.771143,259162.142857
3,00K,53.580857,297603.714286
4,00L,77.890571,325828.428571


In [11]:
baselines = rct_ccgs.merge(pop, on="pct_id").loc[rct_ccgs["period"]=="baseline"]

baselines = baselines.groupby(["joint_id","allocation"]).sum().reset_index()
baselines["calc_value"] = baselines["numerator"]/baselines["denominator"]
baselines["items_thou"] = baselines["items"]/baselines["denominator"]
baselines["percent_over_65"] = 100*baselines["over_65"]/baselines["denominator"]

# group over control and intervention groups:
baselines.groupby("allocation").agg({
    "joint_id":"count",
    "percent_over_65":["mean","std"],
    "denominator":["mean","std"],
    "items_thou":["mean","std"],
    "calc_value":["mean","std"]
    }).round(1)


joint_id percent_over_65      denominator        items_thou        \
              count            mean  std        mean    std       mean   std   
allocation                                                                     
I                20            18.9  4.8       330.9  150.4       32.2  10.9   
con              20            20.4  3.5       259.3  126.0       34.6  12.0   

           calc_value         
                 mean    std  
allocation                    
I              1164.9  140.4  
con            1282.2  231.9

# Primary Outcome P1

Cost per 1,000 patients for all 18 pre-specified “low-priority” treatments combined, between intervention and control groups, assessed by applying a multivariable linear regression model.


In [12]:
out = rct_agg_6m.groupby("allocation").agg({"joint_id":"nunique",
                                                "baseline_calc_value":{"mean","std"},
                                                "follow_up_calc_value":{"mean","std"}})

out["change"] = out[("follow_up_calc_value","mean")] - out[("baseline_calc_value","mean")]

display(out.sort_index(ascending=False))

formula = ('data["follow_up_calc_value"] '
           '~ data["baseline_calc_value"] + intervention')
compute_regression(rct_agg_6m, formula=formula)

joint_id baseline_calc_value             follow_up_calc_value  \
            nunique                mean         std                 mean   
allocation                                                                 
con              20         1282.219714  231.894283          1069.037306   
I                20         1164.939075  140.428957           959.946036   

                            change  
                   std              
allocation                          
con         205.768542 -213.182408  
I           146.688510 -204.993039

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     data["follow_up_calc_value"]   R-squared:                       0.650
Model:                                      OLS   Adj. R-squared:                  0.631
Method:                           Least Squares   F-statistic:                     34.41
Date:                          Thu, 19 Nov 2020   Prob (F-statistic):           3.61e-09
Time:                                  11:29:43   Log-Likelihood:                -244.01
No. Observations:                            40   AIC:                             494.0
Df Residuals:                                37   BIC:                             499.1
Df Model:                                     2                                         
Covariance Type:                      nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                     130.9518    124.314      1.053      0.299    -120.932     382.836
data["baseline_calc_value"]     0.7316      0.095      7.705      0.000       0.539       0.924
intervention                  -23.2875     37.191     -0.626      0.535     -98.643      52.068
==============================================================================
Omnibus:                        3.987   Durbin-Watson:                   1.940
Prob(Omnibus):                  0.136   Jarque-Bera (JB):                3.026
Skew:                          -0.536   Prob(JB):                        0.220
Kurtosis:                       2.183   Cond. No.                     8.76e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.76e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Primary Outcome P2 
ITEMS per 1,000 patients for all 18 pre-specified “low-priority” treatments combined, between intervention and control groups, assessed by applying a multivariable linear regression model.


In [13]:
out = rct_agg_6m.groupby("allocation").agg({"joint_id":"nunique",
                                                "baseline_items_thou":{"mean","std"},
                                                "follow_up_items_thou":{"mean","std"}})

out["change"] = out[("follow_up_items_thou","mean")] - out[("baseline_items_thou","mean")]

display(out.sort_index(ascending=False))

formula = ('data["follow_up_items_thou"] '
           '~ data["baseline_items_thou"] + intervention')
compute_regression(rct_agg_6m, formula=formula)

joint_id baseline_items_thou            follow_up_items_thou  \
            nunique                mean        std                 mean   
allocation                                                                
con              20           34.583888  12.036566            28.044108   
I                20           32.156348  10.872595            25.916282   

                         change  
                  std            
allocation                       
con         10.023813 -6.539780  
I            9.100943 -6.240065

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     data["follow_up_items_thou"]   R-squared:                       0.827
Model:                                      OLS   Adj. R-squared:                  0.818
Method:                           Least Squares   F-statistic:                     88.39
Date:                          Thu, 19 Nov 2020   Prob (F-statistic):           8.08e-15
Time:                                  11:29:43   Log-Likelihood:                -111.27
No. Observations:                            40   AIC:                             228.5
Df Residuals:                                37   BIC:                             233.6
Df Model:                                     2                                         
Covariance Type:                      nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       1.8294      2.185      0.837      0.408      -2.598       6.257
data["baseline_items_thou"]     0.7580      0.057     13.192      0.000       0.642       0.874
intervention                   -0.2877      1.292     -0.223      0.825      -2.906       2.331
==============================================================================
Omnibus:                       29.321   Durbin-Watson:                   2.155
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               69.210
Skew:                          -1.826   Prob(JB):                     9.36e-16
Kurtosis:                       8.309   Cond. No.                         124.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Sensitivity Analysis

Some CCGs did not successfully receive the intervention.

In [15]:
visit = pd.read_csv(os.path.join('..','data','allocated_ccgs_visit_timetable.csv'))
visit["flag"] = np.where(visit["date"].str.len()>0,1,0)

data2 = rct_agg_6m.merge(visit, on="joint_id", how="left").drop("date", axis=1)
data2["flag"] = data2["flag"].fillna(0).astype("int")

out = data2.groupby("flag").agg({"joint_id":"nunique",
                                 "baseline_calc_value":{"mean","std"},
                                 "follow_up_calc_value":{"mean","std"}})

out["change"] = out[("follow_up_calc_value","mean")] - out[("baseline_calc_value","mean")]

display(out)


formula = ('data["follow_up_calc_value"] '
           '~ data["baseline_calc_value"] + flag')
compute_regression(data2, formula=formula)

joint_id baseline_calc_value             follow_up_calc_value  \
      nunique                mean         std                 mean   
flag                                                                 
0          20         1282.219714  231.894283          1069.037306   
1          20         1164.939075  140.428957           959.946036   

                      change  
             std              
flag                          
0     205.768542 -213.182408  
1     146.688510 -204.993039

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     data["follow_up_calc_value"]   R-squared:                       0.650
Model:                                      OLS   Adj. R-squared:                  0.631
Method:                           Least Squares   F-statistic:                     34.41
Date:                          Thu, 19 Nov 2020   Prob (F-statistic):           3.61e-09
Time:                                  11:42:18   Log-Likelihood:                -244.01
No. Observations:                            40   AIC:                             494.0
Df Residuals:                                37   BIC:                             499.1
Df Model:                                     2                                         
Covariance Type:                      nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                     130.9518    124.314      1.053      0.299    -120.932     382.836
data["baseline_calc_value"]     0.7316      0.095      7.705      0.000       0.539       0.924
flag                          -23.2875     37.191     -0.626      0.535     -98.643      52.068
==============================================================================
Omnibus:                        3.987   Durbin-Watson:                   1.940
Prob(Omnibus):                  0.136   Jarque-Bera (JB):                3.026
Skew:                          -0.536   Prob(JB):                        0.220
Kurtosis:                       2.183   Cond. No.                     8.76e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.76e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
out = data2.groupby("flag").agg({"joint_id":"nunique",
                                 "baseline_items_thou":{"mean","std"},
                                 "follow_up_items_thou":{"mean","std"}})

out["change"] = out[("follow_up_items_thou","mean")] - out[("baseline_items_thou","mean")]

display(out)

formula = ('data["follow_up_items_thou"] '
           '~ data["baseline_items_thou"] + flag')
compute_regression(data2, formula=formula)

joint_id baseline_items_thou            follow_up_items_thou             \
      nunique                mean        std                 mean        std   
flag                                                                           
0          20           34.583888  12.036566            28.044108  10.023813   
1          20           32.156348  10.872595            25.916282   9.100943   

        change  
                
flag            
0    -6.539780  
1    -6.240065

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     data["follow_up_items_thou"]   R-squared:                       0.827
Model:                                      OLS   Adj. R-squared:                  0.818
Method:                           Least Squares   F-statistic:                     88.39
Date:                          Thu, 19 Nov 2020   Prob (F-statistic):           8.08e-15
Time:                                  11:42:22   Log-Likelihood:                -111.27
No. Observations:                            40   AIC:                             228.5
Df Residuals:                                37   BIC:                             233.6
Df Model:                                     2                                         
Covariance Type:                      nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       1.8294      2.185      0.837      0.408      -2.598       6.257
data["baseline_items_thou"]     0.7580      0.057     13.192      0.000       0.642       0.874
flag                           -0.2877      1.292     -0.223      0.825      -2.906       2.331
==============================================================================
Omnibus:                       29.321   Durbin-Watson:                   2.155
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               69.210
Skew:                          -1.826   Prob(JB):                     9.36e-16
Kurtosis:                       8.309   Cond. No.                         124.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""